In [48]:
import hashlib
import io
import logging
import os
from os import walk
import pickle
import PIL.Image
import time

from object_detection.utils import dataset_util

import tensorflow as tf
from tensorflow import keras
print('Tensorflow version: ', tf.__version__)
print("Eager execution: ", tf.executing_eagerly())
print("Listing CPUs: ", tf.config.list_physical_devices('CPU'))
print("Listing GPUs: ", tf.config.list_physical_devices('GPU'))

DATA = "./raw_dataset"
PNG_FOLDER = "png_img"
LABELS_FOLDER = "labels"
IDS_FOLDER = "./ids"
OUTPUT_PATH = "./dataset/dataset.record"
IMG_TYPE = "png"

Tensorflow version:  2.1.0
Eager execution:  True
Listing CPUs:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
Listing GPUs:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [49]:
# LISTING ALL EXAMPLES FROM THE IMAGE FOLDER
img_path = os.path.join(DATA, PNG_FOLDER)
images = []
for (dirpath, dirnames, filenames) in walk(img_path):
    for filename in filenames:
        splitted = filename.split('.')
        if len(splitted) and splitted[1].lower()=="png":
            images.append([splitted[0], splitted[1]])

In [73]:
def create_tf_example(img_file, annotation, map_id):
    ## LOADING IMAGE
    filename = img_file[0]+"."+img_file[1]
    img_path = os.path.join(DATA, PNG_FOLDER, filename)
    with tf.io.gfile.GFile(img_path, 'rb') as fid:
        encoded_image_data = fid.read()
    # Checking format
    encoded_image_io = io.BytesIO(encoded_image_data)
    image = PIL.Image.open(encoded_image_io)
    if image.format != 'PNG':
        raise ValueError('Image format not PNG')
    # Hashing
    key = hashlib.sha256(encoded_image_data).hexdigest()
    
    ## GENERAL FEATURES
    width, height = image.size
    image_format = IMG_TYPE.encode('utf8') # b'jpeg' or b'png'
    
    ## DEFINING BOUNDING BOXES
    xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [] # List of normalized right x coordinates in bounding box
    ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [] # List of normalized bottom y coordinates in bounding box
    classes_text = [] # List of string class name of bounding box (1 per box)
    classes = [] # List of integer class id of bounding box (1 per box)
    for bb in annotation:
        xmins.append(bb["xmin"]/width)
        xmaxs.append(bb["xmax"]/width)
        ymins.append(bb["ymin"]/height)
        ymaxs.append(bb["ymax"]/height)
        classes_text.append(bb["taxon"].encode('utf8'))
        classes.append(map_id[bb["taxon"]])
        
        
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename.encode('utf8')),
        'image/source_id': dataset_util.bytes_feature(filename.encode('utf8')),
        'image/key/sha256': dataset_util.bytes_feature(key.encode('utf8')),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [74]:
# CREATE TF RECORD FILE
id_map = pickle.load(open(os.path.join(DATA, IDS_FOLDER, "0000.pickle"), "rb" ))
writer = tf.io.TFRecordWriter(OUTPUT_PATH)
for image in images:
    annotation_path = os.path.join(DATA, LABELS_FOLDER, name+".pickle")
    if os.path.isfile(annotation_path):
        annotation = pickle.load(open(annotation_path, "rb"))
        tf_example = create_tf_example(image, annotation, id_map)
        writer.write(tf_example.SerializeToString())
    else:
        print("ERROR: Image ", name, ".", extension, " has no associated annotation file. Ignoring image.")
writer.close()

1000 1000 b'png'
[0.655, 0.492, 0.0, 0.786, 0.956, 0.205, 0.004, 0.583, 0.45, 0.304, 0.732] [0.771, 0.566, 0.131, 0.844, 0.999, 0.247, 0.11, 0.661, 0.528, 0.378, 0.992] [0.0, 0.462, 0.0, 0.765, 0.05, 0.586, 0.503, 0.582, 0.75, 0.542, 0.449] [0.403, 0.61, 0.293, 0.859, 0.33, 0.774, 0.545, 0.788, 0.999, 0.914, 0.541]
[b'GSTA', b'GACC', b'NEDU', b'HVEN', b'NCAR', b'DMON', b'DENS', b'RUNI', b'NASP', b'PSIN', b'PTLA']
[347, 103, 287, 56, 157, 3, 328, 360, 31, 106, 353]
1000 1000 b'png'
[0.655, 0.492, 0.0, 0.786, 0.956, 0.205, 0.004, 0.583, 0.45, 0.304, 0.732] [0.771, 0.566, 0.131, 0.844, 0.999, 0.247, 0.11, 0.661, 0.528, 0.378, 0.992] [0.0, 0.462, 0.0, 0.765, 0.05, 0.586, 0.503, 0.582, 0.75, 0.542, 0.449] [0.403, 0.61, 0.293, 0.859, 0.33, 0.774, 0.545, 0.788, 0.999, 0.914, 0.541]
[b'GSTA', b'GACC', b'NEDU', b'HVEN', b'NCAR', b'DMON', b'DENS', b'RUNI', b'NASP', b'PSIN', b'PTLA']
[347, 103, 287, 56, 157, 3, 328, 360, 31, 106, 353]
1000 1000 b'png'
[0.655, 0.492, 0.0, 0.786, 0.956, 0.205, 0.00

In [40]:
print(pickle.load(open(os.path.join(DATA, LABELS_FOLDER, "0000.pickle"), "rb")))

[{'taxon': 'DPAR', 'xmin': 21, 'ymin': 111, 'xmax': 199, 'ymax': 421}, {'taxon': 'SRAD', 'xmin': 306, 'ymin': 922, 'xmax': 364, 'ymax': 999}, {'taxon': 'NASP', 'xmin': 800, 'ymin': 0, 'xmax': 908, 'ymax': 316}, {'taxon': 'DMON', 'xmin': 0, 'ymin': 472, 'xmax': 57, 'ymax': 692}, {'taxon': 'NBRG', 'xmin': 294, 'ymin': 365, 'xmax': 638, 'ymax': 429}, {'taxon': 'ECES', 'xmin': 284, 'ymin': 0, 'xmax': 564, 'ymax': 51}, {'taxon': 'CMEN', 'xmin': 526, 'ymin': 678, 'xmax': 658, 'ymax': 812}, {'taxon': 'NVEN', 'xmin': 736, 'ymin': 745, 'xmax': 810, 'ymax': 999}, {'taxon': 'BNEG', 'xmin': 417, 'ymin': 547, 'xmax': 483, 'ymax': 801}]
